In [1]:
import tensorflow as tf;
import numpy as np;
from tensorly import decomposition;
import tensorly;

/home/aman/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/aman/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/aman/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/aman/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWa

In [2]:
tf.enable_eager_execution()

In [3]:
def base(v1,v2,v3):
    ret = np.zeros((len(v1),len(v2),len(v3)))
    return ret;

def expand(v1,v2,v3):
    b = base(v1,v2,v3)
    for i in range(len(v1)):
        for j in range(len(v2)):
            for k in range(len(v3)):
                b[i][j][k] = v1[i] * v2[j] * v3[k]
    return b

In [4]:
tf.__version__

'1.14.0'

In [5]:
init_factors = ([1,2],[0,1,1,0],[2,3,4])
init_arr = expand(init_factors[0],init_factors[1], init_factors[2])

In [6]:
init_arr

array([[[0., 0., 0.],
        [2., 3., 4.],
        [2., 3., 4.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [4., 6., 8.],
        [4., 6., 8.],
        [0., 0., 0.]]])

In [7]:
tensorly.set_backend('numpy')
t_cp = decomposition.parafac(init_arr,1)

In [8]:
t_cp.factors

[array([[ -7.61577311],
        [-15.23154621]]), array([[0.        ],
        [0.70710678],
        [0.70710678],
        [0.        ]]), array([[-0.37139068],
        [-0.55708601],
        [-0.74278135]])]

In [9]:
get_expand_arr(t_cp.factors)

NameError: name 'get_expand_arr' is not defined

In [10]:
indices = [[1,0,0],[1,0,1],[1,0,2],[1,1,0],[1,1,1],[1,1,2],[2,0,0],[2,0,1],[2,0,2],[2,1,0],[2,1,1],[2,1,2],[3,0,0],[3,0,1],[3,0,2],[3,1,0],[3,1,1],[3,1,2]]
values = [16.0, 12.0, 10.0, 64.0, 48.0, 40.0, 24.0, 18.0, 15.0, 96.0, 72.0, 60.0, 56.0, 42.0, 35.0, 224.0, 168.0, 140.0]
shape = [4,2,3]

st = tf.SparseTensor(indices=indices, values=values, dense_shape=shape)

In [11]:
tensorly.get_backend()

'numpy'

In [12]:
tensorly.set_backend('tensorflow')

In [13]:
dt = tf.constant([[[0., 0., 0.],
        [2., 3., 4.],
        [2., 3., 4.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [4., 6., 8.],
        [4., 6., 8.],
        [0., 0., 0.]]])

In [14]:
dt_cp = decomposition.parafac(dt,1)

In [15]:
dt_cp.factors

[<tf.Tensor: id=449, shape=(2, 1), dtype=float32, numpy=
 array([[ -7.615774],
        [-15.231548]], dtype=float32)>,
 <tf.Tensor: id=532, shape=(4, 1), dtype=float32, numpy=
 array([[0.       ],
        [0.7071067],
        [0.7071067],
        [0.       ]], dtype=float32)>,
 <tf.Tensor: id=615, shape=(3, 1), dtype=float32, numpy=
 array([[-0.37139073],
        [-0.55708605],
        [-0.74278146]], dtype=float32)>]

In [16]:
import tensorly.contrib.sparse.decomposition as sp_decomp
#http://tensorly.org/dev/modules/generated/tensorly.contrib.sparse.decomposition.parafac.html#tensorly.contrib.sparse.decomposition.parafac

In [17]:
import sparse

In [18]:
#tensorly.set_backend('tensorflow') 
#ValueError: Unknown backend name 'tensorflow', known backends are ['numpy']
#unsupported operand type(s) for ** or pow(): 'SparseTensor' and 'int'
#tensorly.set_backend('numpy')
#sp_decomp.parafac(st,1) #doesn't work for tensorflow

In [19]:
#tensorly.set_backend('numpy')
#sp_decomp.parafac(init_arr,1)
#ValueError: All arrays must be instances of SparseArray.

In [20]:
nd = 3
dims = (4,2,3)

In [21]:
from tensorly.tenalg import khatri_rao
from tensorly.base import unfold

def hadamard(m1, m2):
    if m1.shape != m2.shape:
        return None
    else:
        ret = np.ndarray(m1.shape)
        for i in range(len(m1)):
            for j in range(len(m1[i])):
                ret[i][j] = m1[i][j] * m2[i][j]
        return ret

def gen_had(matrices, skip_matrix = -1):
    #matrices is a list of matrices
    shp = matrices[0].shape
    for m in matrices: 
        if m.shape != shp:
            return None
    ret = np.ndarray(shp)
    for i in range(shp[0]):
        for j in range(shp[1]):
            prod = 1
            for num,k in enumerate(matrices):
                if(num != skip_matrix):
                    prod *= k[i][j]
                    if prod == 0: #just a lil optimization
                        break;
            ret[i][j] = prod
    return ret
#Doing numpy.ndarray.T returns the transpose
#Doing numpy.matmul(m1,m2) multiples m1 w m2
#Doing unfold(X, i) does a mode i unfolding of tensor X
#Doing khatri_rao(m1,m2) does khatri rao product of m1, m2
#Doing hadamard(m1,m2) does hadamard product of m1, m2
#How to do moore-penrose pseudoinverse?

In [22]:
def calc_indices(t, dims, n):
    #t is the target row we want (zero indexed)
    #dims is the dimensions of the tensor
    #n is the factor matrix we dont want to include
    
    #first get cumulative products on our factors
    prods = [1]
    nd = len(dims)
    for i in range(nd-1,-1,-1):
        if i!=n:
            p = prods[-1] * dims[i]
            prods.append(p)
    prods.reverse()
    prods = prods[1:]
    #now we have our product of dimensions array
    print("Product of dimensions, excluding dimension {}: {}".format(n, prods))
    ret = [] #the coefficients of the thingy we want
    for i in range(nd - 1):
        n = t // prods[i];
        ret.append(n)
        t -= (prods[i] * n)
    return ret

In [23]:
calc_indices(21,[2,4,7,3],2) #should give us [1,3,0]

Product of dimensions, excluding dimension 2: [12, 3, 1]


[1, 3, 0]

In [24]:
def KR_at_ij(factors, i, j, dims, n):
    #dims is the list of dimensions, factors is the actual factor matrix, 
    #factors is the list of all factor matrices
    product = 1
    i_vals = calc_indices(i, dims, n)
    for num, factor in enumerate(factors):
        if num == n:
            pass
        if num < n:
            product *= factors[num][i_vals[num]][j]
        if num > n:
            product *= factors[num][i_vals[num-1]][j]
    return product

In [25]:
def find_index_unfolded(dims, n, idx):
    #finds the coordinates of the unfolded tensor element given the coordinates of the not-unfolded tensor
    j = 0;
    nd = len(dims)
    prod = 1;
    for i in range(nd):
        if i != n:
            j += idx[i] * prod
            prod *= dims[i]
    return (idx[n], j)

In [26]:
print(find_index_unfolded([3,4,2], 0, [1,2,1])) #(1,6)
print(find_index_unfolded([3,4,2], 1, [1,2,1])) #(2,4)
print(find_index_unfolded([3,4,2], 2, [1,2,1])) #(1,7)

(1, 6)
(2, 4)
(1, 7)


In [31]:
def MTTKRP(X, factors, n,rank,dims):
    #X is the tensorflow SPTensor, factors is a list of 
    '''
    Matricized Tensor Times Khatri Rao Product in place:
    go through all the indices in X, and figure out their corresponding place in the mode n unfolding
    Also, given all the factors, we need to figure out where they belong in the khatri rao product
    and then we hvae to multiply
    good news is that matrix multiplication is additive so we can just kinda do stuff
    unfolded tensor is of size: 
        I_n x (I_1 * ... I_n-1 * I_n+1 * ... I_N)
    The khatri rao product is of size:
        (I_1 * ... I_n-1 * I_n+1 * ... I_N) * rank
    '''
    # we will go through all of the indices on the tensor
    # figure out the matricized index: get X[i,j], get KR[j,0]...KR[j,r-1]
    # output[i,0] += X[i,j] * KR[j,0]
    # output[i,1] += X[i,j] * KR[j,1]...
    output = np.zeros((dims[n],rank))
#     indices = X.indices.numpy()
#     values = X.values.numpy()
    for i,idx in enumerate(X.indices.numpy()):
        ij = find_index_unfolded(dims, n, idx)
        print("{}\t{} {} {}".format(i,idx,X.values.numpy()[i],ij))
        val = X.values.numpy()[i]
        for r in range(rank):
            output[ij[0]][r] += val * KR_at_ij(factors,ij[1],r,dims,n)
    

In [32]:
MTTKRP(st,"hello", 0,2,[4,2,3])

0	[1 0 0] 16.0 (1, 0)
Product of dimensions, excluding dimension 0: [3, 1]


TypeError: can't multiply sequence by non-int of type 'str'

In [81]:
tf.sparse.to_dense(st)

<tf.Tensor: id=783, shape=(4, 2, 3), dtype=float32, numpy=
array([[[  0.,   0.,   0.],
        [  0.,   0.,   0.]],

       [[ 16.,  12.,  10.],
        [ 64.,  48.,  40.]],

       [[ 24.,  18.,  15.],
        [ 96.,  72.,  60.]],

       [[ 56.,  42.,  35.],
        [224., 168., 140.]]], dtype=float32)>

In [62]:
st.values.numpy()[2]

10.0

In [73]:
print(st.indices.numpy()[1])

[1 0 1]


In [27]:
def cp_als(X, r):
    factors = []
    for i in range(nd):
        factors.append(np.ones((dimensions[i],r)))
    for repeats in range(3):                   #how many iterations are we doing
        for n in range(nd):
            l_h = [mul(f.T,f) for f in factors] 
            V = gen_had(l_h, skip_matrix=n)    #gets hadamard product of all mult(f[i].T,f[i])
            Vinv = np.linalg.pinv(V)           #do pseudoinverse in case V is singular
            mk = MTTKRP(X, factors, n,r,dims)
            factors[n] = mul(mk,Vinv)
    return factors

In [50]:
len(st.values.numpy())

18

# Things necessary to make this algorithm work for tf.SparseTensor

**Note:** factors are dense, so they can be stored with numpy.ndarrays.

**Note:**
* dimensions of V: rank * rank
* dim of Vinv: rank * rank
* dim of X_u: $I_n * (\Pi I_i)/I_n$, note, this is just a rearrangement of our initial SparseTensor, so this will be sparse
* dim of KR: 
* dim of tmp1 = $(\Pi I_i)/I_n * rank$, 
* X_u * temp1 = (I_n * something) * (something, R)
* dim of tmp2: I_n * rank (makes sense because this is the literal definition of factor size)


**Note:** 

In [28]:
nd = 3
dimensions=(2,4,3)
f = cp_als(init_arr, 1)

NameError: name 'mul' is not defined

In [29]:
f

NameError: name 'f' is not defined

In [30]:
def get_expand_arr(factors):
    nd = len(f)
    dimensions = [len(factor) for factor in f]
    dims = tuple(dimensions)
    base = ret = np.zeros(dims)
    
    def exp(num, idx, prod):
        if prod == 0:
            return
        if num == nd:
            base.itemset(tuple(idx), prod)
            return
        else:
            factor = factors[num]
            for i in range(len(factor)):
                exp(num+1, idx + [i], prod * factor[i])
    
    exp(0,[],1)
    return base

In [31]:
get_expand_arr(f) #off by a factor of 2??

NameError: name 'f' is not defined

In [32]:
init_arr

array([[[0., 0., 0.],
        [2., 3., 4.],
        [2., 3., 4.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [4., 6., 8.],
        [4., 6., 8.],
        [0., 0., 0.]]])